use this line to instal depenencies if it doesnt work try to build the virtual env from the reqirements.txt

In [21]:
import json
import os
from getpass import getpass
from urllib.request import urlopen
import logging
import time

from langchain.vectorstores import ElasticsearchStore
from langchain import HuggingFacePipeline
from langchain_core.documents.base import Document 

from transformers import AutoTokenizer, pipeline
from huggingface_hub import notebook_login
from elasticsearch.exceptions import ConnectionTimeout
from transformers import AutoTokenizer, pipeline, AutoModelForCausalLM


In [3]:
elastic_index_name = "gemma-rag"

In [4]:
url = "https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/datasets/workplace-documents.json"

response = urlopen(url)

workplace_docs = json.loads(response.read())

### Preprocessing the document and splitting into fixed size chunks
fix size is easier to debug if something goes bad, i selected a simple text with minimal amount of footers/headers but still needed to clear it from excessive carrige return/new line characters. Decided to leave the quotation marks in the text, as well as some gibberish at the begining end (it is fraction of the total dataset)

In [8]:
import re

target_url = "https://www.gutenberg.org/cache/epub/2641/pg2641.txt"

target_response = urlopen(target_url)

content = target_response.read().decode("utf-8")

cleaned_text = re.sub(r'[\n\r]+', ' ', content)
cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
chunk = 100
chunked_text = [cleaned_text[i:i+chunk] for i in range(0, len(cleaned_text)-chunk, chunk)]
docs = [Document(page_content = chunk) for chunk in chunked_text]
len(docs), docs[:5]

(3917,
 [Document(page_content='\ufeffThe Project Gutenberg eBook of A Room with a View This ebook is for the use of anyone anywhere in t'),
  Document(page_content='he United States and most other parts of the world at no cost and with almost no restrictions whatso'),
  Document(page_content='ever. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License in'),
  Document(page_content='cluded with this ebook or online at www.gutenberg.org. If you are not located in the United States, '),
  Document(page_content='you will have to check the laws of the country where you are located before using this eBook. Title:')])

### Indexing using ElasticSearch, due to some error problems on their side tryign multiple times

State of the art, easier to scale-up/down (just rent more compute units from the elastisearch) and monitor durin deployment (they have nice dashboard to monitor the progress and various jobs status). For the retrieval model I used ELSER (Elastic Learned Sparse EncodeR) with the newest version 2. Compared to say BERT is better for indexing large amounts of data (like books we are deadlin with).

### Cant generate embedding for the whol sequence since I run out of credits.....

In [10]:
logger = logging.getLogger(__name__)

max_attempts = 10
for attempt in range(max_attempts):
    try:
        embedding_space = ElasticsearchStore.from_documents(
            docs[:2000],
            es_cloud_id=ELASTIC_CLOUD_ID,
            es_api_key=ELASTIC_API_KEY,
            index_name=elastic_index_name,
            timeout = 30,
            strategy=ElasticsearchStore.SparseVectorRetrievalStrategy(
                model_id=".elser_model_2"
            ),
        )
        break
    except ConnectionTimeout:  
        if attempt < max_attempts - 1:
            logger.warning(f"Connection timed out. Retrying ({attempt + 1}/{max_attempts})...")
            time.sleep(5)
        else:
            logger.error("Connection failed after several attempts. Please try again later.")

In [11]:
question = 'what city is the book taking place?'
embedding_space.as_retriever(search_kwargs={"k": 5}).invoke(question)

docs = embedding_space.as_retriever(search_kwargs={"k": 5}).invoke(question)
seen = set()
for doc in docs:
    if doc.page_content not in seen:
        seen.add(doc.page_content)
    
context = ''.join([doc.page_content for doc in docs ])
context

'The scene is laid in Florence.” “What fun, Cecil! Read away. Come, Mr. Emerson, sit down after all yThe scene is laid in Florence.” “What fun, Cecil! Read away. Come, Mr. Emerson, sit down after all yThe scene is laid in Florence.” “What fun, Cecil! Read away. Come, Mr. Emerson, sit down after all ynkle of church bells. The garden of Windy Corners was deserted except for a red book, which lay sunnnkle of church bells. The garden of Windy Corners was deserted except for a red book, which lay sunn'

### Hugging Face login
using Hf is nice because we can quikcly experiment with differnt models for our applicaton, its also SOTA

In [12]:
notebook_login()

### Initialize the tokenizer and the model (`google/gemma-2b-it`)

Lightweight, state-of-the-art open models from Google, built from the same research and technology used to create the Gemini models. Since these are pretty new and hot right now I chose them over variation of different old and tested methods like T5, BART, XLNet Albert, RoBERT with text-generation variation implemtation in hugging face. For the parameter tunning, I set the temperature to 0.7 experimentaly it seemed to me to give a bit better results on the dataset. Since it runs localy on my PC weights must be top 8GB in RAM which limits possible models. ON smaller model inference time is also quicker so its simpler to experiment with.

In [22]:
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it")
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
)

llm = HuggingFacePipeline(
    pipeline=pipe,
    model_kwargs={"temperature": 0.7},
)

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### Geerating the answer text based on the RAG output
use the augmented prompt with the text-generation selected model, set long max_length in case user would be interested in the summarization of long passages or descriptions often found in books.

In [28]:
prompt = f'Answer the question in the following quotation marks: "{question}" information provided in the following context surounded by the quotation marks: "{context}"'
input_ids = tokenizer.encode(prompt, return_tensors="pt")
outputs = model.generate(input_ids, max_length=200)
output_ids = outputs[:, input_ids.shape[-1]:]
output_text = tokenizer.decode(output_ids[0])
output_text

'\n\nThe book is taking place in Florence.<eos>'

### Evaluation Metrics

For metric I could use supervised metrics (e.g. I would manualy craft 20 questions regardin the text with my answer and they dry to detect keyword in chatboot solution to see if it found them, or even read through them and qualitatively assume responses) and more unsupervised techniques e.g. ensembling couple of models, then prompting them with the same question and then clustering their answers together (assuming that consensus is right) and benchamr against this.